In [1]:
import os
from dotenv import load_dotenv # requires python-dotenv
load_dotenv("./../config/credentials_my.env")

print(f'Project Connection String: <...{os.environ["PROJECT_CONNECTION_STRING"][-30:]}>')

Project Connection String: <...mai04-rg;mmai-hub04-prj01-fvye>


In [3]:
from azure.ai.projects.models import azure

In [ ]:
import os
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import FilePurpose
from azure.identity import DefaultAzureCredential

In [ ]:
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=os.environ["PROJECT_CONNECTION_STRING"]
)

In [ ]:
project_client.agents.list_agents()

In [ ]:
connections = project_client.connections.list()
for connection in connections:
    print(connection)

In [ ]:
file = project_client.agents.upload_file_and_poll(
    file_path="./product_info_1.md", purpose=FilePurpose.AGENTS
)